In [ ]:
import numpy as np
import tensorflow as tf
import random as rd

from matplotlib import pyplot

from sklearn.model_selection import StratifiedKFold

import keras_metrics as km 

from keras.models import Model
from keras.utils import to_categorical
from keras.models import clone_model
from keras.callbacks import EarlyStopping

import import_ipynb
from Productive_TimeHistory import TimeHistory

from keras.models import load_model

In [ ]:
#model = Kearas model
#softmax = boolean, true if the model uses softmaxe in the output layer otherwise false
#outputpath = path were the output csv should be stored
#outputname = name to use for the output csv
def cross_val(model, nrepochs, nrbatch, optimizer, lossfunction, train_data, train_labels, softmax, outputpath, outputname):
    import datetime
    
    seed = 7
    np.random.seed(seed)
    tf.set_random_seed(seed)
    rd.seed(seed)
    
    #needed for softmax-activation
    if softmax:
        labelstrain = to_categorical(train_labels)
    else:
        labelstrain = train_labels

    #Have to convert the data to an np.arry, to use the indices generated by StratifiedKFold
    crossvaldata = np.array(train_data)
    crossvallabels = np.array(labelstrain)

    #Define 10-fold cross validation test harness with StratifiedKFold
    skfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)

    #Define variabels to store the scores for every fold
    accscores = []
    precisionscores = []
    recallscores = []
    f1scores = []
    

    #Fold number, needed for stoing to csv
    nrfold = 1

    #Cross validation
    for train, test in skfold.split(train_data, train_labels):
        time_callback = TimeHistory() 
        
        foldmodel = clone_model(model)
        
        #evaluate the positive class (=ironic data)
        precision = km.binary_precision(label=1)
        recall = km.binary_recall(label=1) 
        f1 = km.f1_score(label=1)
        
        foldmodel.compile(loss = lossfunction,
              optimizer=optimizer,
              metrics=['accuracy',precision, recall, f1])

        #Train the model with validation data splited through cross validation
        hist= foldmodel.fit(crossvaldata[train], crossvallabels[train], epochs=nrepochs, batch_size=nrbatch, 
                            validation_data=(crossvaldata[test], crossvallabels[test]), shuffle=False, verbose = 0, callbacks=[time_callback])

        #Append average of the validation scores for the current fold
        accscores.append(np.mean(hist.history['val_acc'])*100)
        precisionscores.append(np.mean(hist.history['val_precision'])*100)
        recallscores.append(np.mean(hist.history['val_recall'])*100)
        f1scores.append(np.mean(hist.history['val_f1_score'])*100)

        print("\n-----Fold "+str(nrfold)+"--------")
        
        #Plot the metrics for every epoch for the current fold
        from matplotlib import pyplot
        %matplotlib inline
        pyplot.figure(figsize=(15,10))
        pyplot.plot(hist.history['acc']) 
        pyplot.plot(hist.history['f1_score'])
        pyplot.plot(hist.history['val_acc']) 
        pyplot.plot(hist.history['val_f1_score'])
        #pyplot.plot(hist.history['loss'])
        pyplot.xticks(np.arange(1,nrepochs))
        pyplot.legend(['Train Accuracy', 'Train F1', 'Validation Accuracy', 'Validation F1', 'Loss'], loc=(1.04,0.5))
        pyplot.show()
        
        #Print the average validation scores for the current fold
        print("Average Accuracy: %.2f%%" % ((np.mean(hist.history['val_acc']))*100))
        print("Average Precision: %.2f%%" % ((np.mean(hist.history['val_precision']))*100))
        print("Average Recall: %.2f%%" % ((np.mean(hist.history['val_recall']))*100))
        print("Average F1: %.2f%%" % ((np.mean(hist.history['val_f1_score']))*100))

        #Store the keras model
        foldmodel.save(outputpath+str(datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S"))+'_'+outputname+
                       '_'+str(nrfold)+'_model.h5')

        #Store the train and validation scores for every epoch for the current fold to csv + train time
        f= open(outputpath+str(datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S"))+'_'+outputname+'_'+
                str(nrfold)+'_train_val_scores.csv','w+')
        f.write('Epoch; Loss; Accuracy; Precision; Recall; F1; Val_loss; Val_Accuracy; Val_Precision; VAl_Recall; Val_F1; Time \n')

        for i in range(len(hist.history['acc'])):
            f.write(str(i+1)+';'+str(hist.history['loss'][i])+';'+str(hist.history['acc'][i])+';'+
                    str(hist.history['precision'][i])+';'+str(hist.history['recall'][i])+';'+
                    str(hist.history['f1_score'][i])+';'+str(hist.history['val_loss'][i])+';'+
                    str(hist.history['val_acc'][i])+';'+str(hist.history['val_precision'][i])+';'+
                    str(hist.history['val_recall'][i])+';'+str(hist.history['val_f1_score'][i])+';'+
                    str(time_callback.times[i])+'\n')

        f.close()

        nrfold+=1
      
    #Plot and print the results of all folds 
    print("\n-------Overallresults-------")

    #Plot the overall Metrics
    pyplot.figure(figsize=(15,10))
    pyplot.plot(accscores) 
    pyplot.plot(precisionscores)
    pyplot.plot(recallscores)
    pyplot.plot(f1scores)
    #pyplot.xticks(np.arange(1,nrfold))
    pyplot.legend(['Accuracy', 'Precision', 'Recall', 'F1'], loc=(1.04,0.5))
    pyplot.show()

    #Print mean, min and max values
    print("Accuracy: Mean = %.2f%% (+/- %.2f%%), Max = %.2f%%, Min =  %.2f%%" % (np.mean(accscores), np.std(accscores), np.max(accscores), np.min(accscores)))
    print("Precision: Mean = %.2f%% (+/- %.2f%%), Max = %.2f%%, Min =  %.2f%%" % (np.mean(precisionscores), np.std(precisionscores), np.max(precisionscores), np.min(precisionscores)))
    print("Recall: Mean = %.2f%% (+/- %.2f%%), Max = %.2f%%, Min =  %.2f%%" % (np.mean(recallscores), np.std(recallscores), np.max(recallscores), np.min(recallscores)))
    print("F1: Mean = %.2f%% (+/- %.2f%%), Max = %.2f%%, Min =  %.2f%%" % (np.mean(f1scores), np.std(f1scores), np.max(f1scores), np.min(f1scores)))
    
    #Write validation scores for every fold to csv
    f= open(outputpath+str(datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S"))+' '+outputname+
            '_validationscores.csv',"w+")
    f.write('Accuracy; Precision; Recall; F1 \n')
    for i in range(len(accscores)):
        f.write(str(accscores[i])+';'+str(precisionscores[i])+';'+str(recallscores[i])+';'+str(f1scores[i])+'\n')

    f.close()

In [ ]:
#model = Kearas model
#softmax = boolean, true if the model uses softmaxe in the output layer otherwise false
#outputpath = path were the output csv should be stored
#outputname = name to use for the output csv
def cross_val_with_earlystopping(model, nrepochs, nrbatch, optimizer, lossfunction, train_data, train_labels, 
                                 softmax, outputpath, outputname, es_monitor, es_patience):
    import datetime
    
    seed = 7
    np.random.seed(seed)
    tf.set_random_seed(seed)
    rd.seed(seed)
    
    #needed for softmax-activation
    if softmax:
        labelstrain = to_categorical(train_labels)
    else:
        labelstrain = train_labels

    #Have to convert the data to an np.arry, to use the indices generated by StratifiedKFold
    crossvaldata = np.array(train_data)
    crossvallabels = np.array(labelstrain)

    #Define 10-fold cross validation test harness with StratifiedKFold
    skfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)

    #Define variabels to store the scores for every fold
    accscores = []
    precisionscores = []
    recallscores = []
    f1scores = []
    

    #Fold number, needed for stoing to csv
    nrfold = 1

    #Cross validation
    for train, test in skfold.split(train_data, train_labels):
        time_callback = TimeHistory() 
        
        es = EarlyStopping(monitor=es_monitor,
                              min_delta=0,
                              patience=es_patience,
                              verbose=0, mode='auto')
        
        foldmodel = clone_model(model)
        
        #evaluate the positive class (=ironic data)
        precision = km.binary_precision(label=1)
        recall = km.binary_recall(label=1) 
        f1 = km.f1_score(label=1)
        
        foldmodel.compile(loss = lossfunction,
              optimizer=optimizer,
              metrics=['accuracy',precision, recall, f1])

        #Train the model with validation data splited through cross validation
        hist= foldmodel.fit(crossvaldata[train], crossvallabels[train], epochs=nrepochs, batch_size=nrbatch, 
                            validation_data=(crossvaldata[test], crossvallabels[test]), shuffle=False, verbose = 1, callbacks=[es, time_callback])

        #Append average of the validation scores for the current fold
        accscores.append(np.mean(hist.history['val_acc'])*100)
        precisionscores.append(np.mean(hist.history['val_precision'])*100)
        recallscores.append(np.mean(hist.history['val_recall'])*100)
        f1scores.append(np.mean(hist.history['val_f1_score'])*100)

        print("\n-----Fold "+str(nrfold)+"--------")
        
        #Plot the metrics for every epoch for the current fold
        from matplotlib import pyplot
        %matplotlib inline
        pyplot.figure(figsize=(15,10))
        pyplot.plot(hist.history['acc']) 
        pyplot.plot(hist.history['f1_score'])
        pyplot.plot(hist.history['val_acc']) 
        pyplot.plot(hist.history['val_f1_score'])
        #pyplot.plot(hist.history['loss'])
        pyplot.xticks(np.arange(1,nrepochs))
        pyplot.legend(['Train Accuracy', 'Train F1', 'Validation Accuracy', 'Validation F1', 'Loss'], loc=(1.04,0.5))
        pyplot.show()
        
        #Print the average validation scores for the current fold
        print("Average Accuracy: %.2f%%" % ((np.mean(hist.history['val_acc']))*100))
        print("Average Precision: %.2f%%" % ((np.mean(hist.history['val_precision']))*100))
        print("Average Recall: %.2f%%" % ((np.mean(hist.history['val_recall']))*100))
        print("Average F1: %.2f%%" % ((np.mean(hist.history['val_f1_score']))*100))

        #Store the keras model
        foldmodel.save(outputpath+str(datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S"))+'_'+outputname+
                       '_'+str(nrfold)+'_model.h5')

        #Store the train and validation scores for every epoch for the current fold to csv + train time
        f= open(outputpath+str(datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S"))+'_'+outputname+'_'+
                str(nrfold)+'_train_val_scores.csv','w+')
        f.write('Epoch; Loss; Accuracy; Precision; Recall; F1; Val_loss; Val_Accuracy; Val_Precision; VAl_Recall; Val_F1; Time \n')

        for i in range(len(hist.history['acc'])):
            f.write(str(i+1)+';'+str(hist.history['loss'][i])+';'+str(hist.history['acc'][i])+';'+
                    str(hist.history['precision'][i])+';'+str(hist.history['recall'][i])+';'+
                    str(hist.history['f1_score'][i])+';'+str(hist.history['val_loss'][i])+';'+
                    str(hist.history['val_acc'][i])+';'+str(hist.history['val_precision'][i])+';'+
                    str(hist.history['val_recall'][i])+';'+str(hist.history['val_f1_score'][i])+';'+
                    str(time_callback.times[i])+'\n')

        f.close()

        nrfold+=1
      
    #Plot and print the results of all folds 
    print("\n-------Overallresults-------")

    #Plot the overall Metrics
    pyplot.figure(figsize=(15,10))
    pyplot.plot(accscores) 
    pyplot.plot(precisionscores)
    pyplot.plot(recallscores)
    pyplot.plot(f1scores)
    #pyplot.xticks(np.arange(1,nrfold))
    pyplot.legend(['Accuracy', 'Precision', 'Recall', 'F1'], loc=(1.04,0.5))
    pyplot.show()

    #Print mean, min and max values
    print("Accuracy: Mean = %.2f%% (+/- %.2f%%), Max = %.2f%%, Min =  %.2f%%" % (np.mean(accscores), np.std(accscores), np.max(accscores), np.min(accscores)))
    print("Precision: Mean = %.2f%% (+/- %.2f%%), Max = %.2f%%, Min =  %.2f%%" % (np.mean(precisionscores), np.std(precisionscores), np.max(precisionscores), np.min(precisionscores)))
    print("Recall: Mean = %.2f%% (+/- %.2f%%), Max = %.2f%%, Min =  %.2f%%" % (np.mean(recallscores), np.std(recallscores), np.max(recallscores), np.min(recallscores)))
    print("F1: Mean = %.2f%% (+/- %.2f%%), Max = %.2f%%, Min =  %.2f%%" % (np.mean(f1scores), np.std(f1scores), np.max(f1scores), np.min(f1scores)))
    
    #Write validation scores for every fold to csv
    f= open(outputpath+str(datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S"))+' '+outputname+
            '_validationscores.csv',"w+")
    f.write('Accuracy; Precision; Recall; F1 \n')
    for i in range(len(accscores)):
        f.write(str(accscores[i])+';'+str(precisionscores[i])+';'+str(recallscores[i])+';'+str(f1scores[i])+'\n')

    f.close()

In [ ]:
#model = Kearas model
#softmax = boolean, true if the model uses softmaxe in the output layer otherwise false
#outputpath = path were the output csv should be stored
#outputname = name to use for the output csv
def cross_val_two_inputs(model, nrepochs, nrbatch, optimizer, lossfunction, train_data, train_data2, train_labels, softmax, outputpath, outputname):
    import datetime
    
    seed = 7
    np.random.seed(seed)
    tf.set_random_seed(seed)
    rd.seed(seed)
    
    #needed for softmax-activation
    if softmax:
        labelstrain = to_categorical(train_labels)
    else:
        labelstrain = train_labels

    #Have to convert the data to an np.arry, to use the indices generated by StratifiedKFold
    crossvaldata = np.array(train_data)
    crossvaldata2 = np.array(train_data2)
    crossvallabels = np.array(labelstrain)

    #Define 10-fold cross validation test harness with StratifiedKFold
    skfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)

    #Define variabels to store the scores for every fold
    accscores = []
    precisionscores = []
    recallscores = []
    f1scores = []
    

    #Fold number, needed for stoing to csv
    nrfold = 1

    #Cross validation
    for train, test in skfold.split(train_data, train_labels):
        time_callback = TimeHistory() 
        
        foldmodel = clone_model(model)
        
        #evaluate the positive class (=ironic data)
        precision = km.binary_precision(label=1)
        recall = km.binary_recall(label=1) 
        f1 = km.f1_score(label=1)
        
        foldmodel.compile(loss = lossfunction,
              optimizer=optimizer,
              metrics=['accuracy',precision, recall, f1])

        #Train the model with validation data splited through cross validation
        hist= foldmodel.fit([crossvaldata[train], crossvaldata2[train]], crossvallabels[train], epochs=nrepochs, batch_size=nrbatch, 
                            validation_data=([crossvaldata[test],crossvaldata2[test]], crossvallabels[test]), shuffle=False, verbose = 0, callbacks=[time_callback])

        #Append average of the validation scores for the current fold
        accscores.append(np.mean(hist.history['val_acc'])*100)
        precisionscores.append(np.mean(hist.history['val_precision'])*100)
        recallscores.append(np.mean(hist.history['val_recall'])*100)
        f1scores.append(np.mean(hist.history['val_f1_score'])*100)

        print("\n-----Fold "+str(nrfold)+"--------")
        
        #Plot the metrics for every epoch for the current fold
        from matplotlib import pyplot
        %matplotlib inline
        pyplot.figure(figsize=(15,10))
        pyplot.plot(hist.history['acc']) 
        pyplot.plot(hist.history['f1_score'])
        pyplot.plot(hist.history['val_acc']) 
        pyplot.plot(hist.history['val_f1_score'])
        #pyplot.plot(hist.history['loss'])
        pyplot.xticks(np.arange(1,nrepochs))
        pyplot.legend(['Train Accuracy', 'Train F1', 'Validation Accuracy', 'Validation F1', 'Loss'], loc=(1.04,0.5))
        pyplot.show()
        
        #Print the average validation scores for the current fold
        print("Average Accuracy: %.2f%%" % ((np.mean(hist.history['val_acc']))*100))
        print("Average Precision: %.2f%%" % ((np.mean(hist.history['val_precision']))*100))
        print("Average Recall: %.2f%%" % ((np.mean(hist.history['val_recall']))*100))
        print("Average F1: %.2f%%" % ((np.mean(hist.history['val_f1_score']))*100))

        #Store the keras model
        foldmodel.save(outputpath+str(datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S"))+'_'+outputname+
                       '_'+str(nrfold)+'_model.h5')

        #Store the train and validation scores for every epoch for the current fold to csv + train time
        f= open(outputpath+str(datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S"))+'_'+outputname+'_'+
                str(nrfold)+'_train_val_scores.csv','w+')
        f.write('Epoch; Loss; Accuracy; Precision; Recall; F1; Val_loss; Val_Accuracy; Val_Precision; VAl_Recall; Val_F1; Time \n')

        for i in range(len(hist.history['acc'])):
            f.write(str(i+1)+';'+str(hist.history['loss'][i])+';'+str(hist.history['acc'][i])+';'+
                    str(hist.history['precision'][i])+';'+str(hist.history['recall'][i])+';'+
                    str(hist.history['f1_score'][i])+';'+str(hist.history['val_loss'][i])+';'+
                    str(hist.history['val_acc'][i])+';'+str(hist.history['val_precision'][i])+';'+
                    str(hist.history['val_recall'][i])+';'+str(hist.history['val_f1_score'][i])+';'+
                    str(time_callback.times[i])+'\n')

        f.close()

        nrfold+=1
      
    #Plot and print the results of all folds 
    print("\n-------Overallresults-------")

    #Plot the overall Metrics
    pyplot.figure(figsize=(15,10))
    pyplot.plot(accscores) 
    pyplot.plot(precisionscores)
    pyplot.plot(recallscores)
    pyplot.plot(f1scores)
    #pyplot.xticks(np.arange(1,nrfold))
    pyplot.legend(['Accuracy', 'Precision', 'Recall', 'F1'], loc=(1.04,0.5))
    pyplot.show()

    #Print mean, min and max values
    print("Accuracy: Mean = %.2f%% (+/- %.2f%%), Max = %.2f%%, Min =  %.2f%%" % (np.mean(accscores), np.std(accscores), np.max(accscores), np.min(accscores)))
    print("Precision: Mean = %.2f%% (+/- %.2f%%), Max = %.2f%%, Min =  %.2f%%" % (np.mean(precisionscores), np.std(precisionscores), np.max(precisionscores), np.min(precisionscores)))
    print("Recall: Mean = %.2f%% (+/- %.2f%%), Max = %.2f%%, Min =  %.2f%%" % (np.mean(recallscores), np.std(recallscores), np.max(recallscores), np.min(recallscores)))
    print("F1: Mean = %.2f%% (+/- %.2f%%), Max = %.2f%%, Min =  %.2f%%" % (np.mean(f1scores), np.std(f1scores), np.max(f1scores), np.min(f1scores)))
    
    #Write validation scores for every fold to csv
    f= open(outputpath+str(datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S"))+' '+outputname+
            '_validationscores.csv',"w+")
    f.write('Accuracy; Precision; Recall; F1 \n')
    for i in range(len(accscores)):
        f.write(str(accscores[i])+';'+str(precisionscores[i])+';'+str(recallscores[i])+';'+str(f1scores[i])+'\n')

    f.close()

In [ ]:
#model = Kearas model
#softmax = boolean, true if the model uses softmaxe in the output layer otherwise false
#outputpath = path were the output csv should be stored
#outputname = name to use for the output csv
def cross_val_three_inputs(model, nrepochs, nrbatch, optimizer, lossfunction, train_data, train_data2, train_data3,
                           train_labels, softmax, outputpath, outputname):
    import datetime
    
    seed = 7
    np.random.seed(seed)
    tf.set_random_seed(seed)
    rd.seed(seed)
    
    #needed for softmax-activation
    if softmax:
        labelstrain = to_categorical(train_labels)
    else:
        labelstrain = train_labels

    #Have to convert the data to an np.arry, to use the indices generated by StratifiedKFold
    crossvaldata = np.array(train_data)
    crossvaldata2 = np.array(train_data2)
    crossvaldata3 = np.array(train_data3)
    crossvallabels = np.array(labelstrain)

    #Define 10-fold cross validation test harness with StratifiedKFold
    skfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)

    #Define variabels to store the scores for every fold
    accscores = []
    precisionscores = []
    recallscores = []
    f1scores = []
    

    #Fold number, needed for stoing to csv
    nrfold = 1

    #Cross validation
    for train, test in skfold.split(train_data, train_labels):
        time_callback = TimeHistory() 
        
        foldmodel = clone_model(model)
        
        #evaluate the positive class (=ironic data)
        precision = km.binary_precision(label=1)
        recall = km.binary_recall(label=1) 
        f1 = km.f1_score(label=1)
        
        foldmodel.compile(loss = lossfunction,
              optimizer=optimizer,
              metrics=['accuracy',precision, recall, f1])

        #Train the model with validation data splited through cross validation
        hist= foldmodel.fit([crossvaldata[train], crossvaldata2[train],  crossvaldata3[train]], crossvallabels[train], epochs=nrepochs, batch_size=nrbatch, 
                            validation_data=([crossvaldata[test],crossvaldata2[test], crossvaldata3[test]], 
                            crossvallabels[test]), shuffle=False, verbose = 0, callbacks=[time_callback])

        #Append average of the validation scores for the current fold
        accscores.append(np.mean(hist.history['val_acc'])*100)
        precisionscores.append(np.mean(hist.history['val_precision'])*100)
        recallscores.append(np.mean(hist.history['val_recall'])*100)
        f1scores.append(np.mean(hist.history['val_f1_score'])*100)

        print("\n-----Fold "+str(nrfold)+"--------")
        
        #Plot the metrics for every epoch for the current fold
        from matplotlib import pyplot
        %matplotlib inline
        pyplot.figure(figsize=(15,10))
        pyplot.plot(hist.history['acc']) 
        pyplot.plot(hist.history['f1_score'])
        pyplot.plot(hist.history['val_acc']) 
        pyplot.plot(hist.history['val_f1_score'])
        #pyplot.plot(hist.history['loss'])
        pyplot.xticks(np.arange(1,nrepochs))
        pyplot.legend(['Train Accuracy', 'Train F1', 'Validation Accuracy', 'Validation F1', 'Loss'], loc=(1.04,0.5))
        pyplot.show()
        
        #Print the average validation scores for the current fold
        print("Average Accuracy: %.2f%%" % ((np.mean(hist.history['val_acc']))*100))
        print("Average Precision: %.2f%%" % ((np.mean(hist.history['val_precision']))*100))
        print("Average Recall: %.2f%%" % ((np.mean(hist.history['val_recall']))*100))
        print("Average F1: %.2f%%" % ((np.mean(hist.history['val_f1_score']))*100))

        #Store the keras model
        foldmodel.save(outputpath+str(datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S"))+'_'+outputname+
                       '_'+str(nrfold)+'_model.h5')

        #Store the train and validation scores for every epoch for the current fold to csv + train time
        f= open(outputpath+str(datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S"))+'_'+outputname+'_'+
                str(nrfold)+'_train_val_scores.csv','w+')
        f.write('Epoch; Loss; Accuracy; Precision; Recall; F1; Val_loss; Val_Accuracy; Val_Precision; VAl_Recall; Val_F1; Time \n')

        for i in range(len(hist.history['acc'])):
            f.write(str(i+1)+';'+str(hist.history['loss'][i])+';'+str(hist.history['acc'][i])+';'+
                    str(hist.history['precision'][i])+';'+str(hist.history['recall'][i])+';'+
                    str(hist.history['f1_score'][i])+';'+str(hist.history['val_loss'][i])+';'+
                    str(hist.history['val_acc'][i])+';'+str(hist.history['val_precision'][i])+';'+
                    str(hist.history['val_recall'][i])+';'+str(hist.history['val_f1_score'][i])+';'+
                    str(time_callback.times[i])+'\n')

        f.close()

        nrfold+=1
      
    #Plot and print the results of all folds 
    print("\n-------Overallresults-------")

    #Plot the overall Metrics
    pyplot.figure(figsize=(15,10))
    pyplot.plot(accscores) 
    pyplot.plot(precisionscores)
    pyplot.plot(recallscores)
    pyplot.plot(f1scores)
    #pyplot.xticks(np.arange(1,nrfold))
    pyplot.legend(['Accuracy', 'Precision', 'Recall', 'F1'], loc=(1.04,0.5))
    pyplot.show()

    #Print mean, min and max values
    print("Accuracy: Mean = %.2f%% (+/- %.2f%%), Max = %.2f%%, Min =  %.2f%%" % (np.mean(accscores), np.std(accscores), np.max(accscores), np.min(accscores)))
    print("Precision: Mean = %.2f%% (+/- %.2f%%), Max = %.2f%%, Min =  %.2f%%" % (np.mean(precisionscores), np.std(precisionscores), np.max(precisionscores), np.min(precisionscores)))
    print("Recall: Mean = %.2f%% (+/- %.2f%%), Max = %.2f%%, Min =  %.2f%%" % (np.mean(recallscores), np.std(recallscores), np.max(recallscores), np.min(recallscores)))
    print("F1: Mean = %.2f%% (+/- %.2f%%), Max = %.2f%%, Min =  %.2f%%" % (np.mean(f1scores), np.std(f1scores), np.max(f1scores), np.min(f1scores)))
    
    #Write validation scores for every fold to csv
    f= open(outputpath+str(datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S"))+' '+outputname+
            '_validationscores.csv',"w+")
    f.write('Accuracy; Precision; Recall; F1 \n')
    for i in range(len(accscores)):
        f.write(str(accscores[i])+';'+str(precisionscores[i])+';'+str(recallscores[i])+';'+str(f1scores[i])+'\n')

    f.close()

In [ ]:
#model = Kearas model
#softmax = boolean, true if the model uses softmaxe in the output layer otherwise false
#outputpath = path were the output csv should be stored
#outputname = name to use for the output csv
def cross_val_four_inputs(model, nrepochs, nrbatch, optimizer, lossfunction, train_data, train_data2, train_data3, train_data4,
                           train_labels, softmax, outputpath, outputname):
    import datetime
    
    seed = 7
    np.random.seed(seed)
    tf.set_random_seed(seed)
    rd.seed(seed)
    
    #needed for softmax-activation
    if softmax:
        labelstrain = to_categorical(train_labels)
    else:
        labelstrain = train_labels

    #Have to convert the data to an np.arry, to use the indices generated by StratifiedKFold
    crossvaldata = np.array(train_data)
    crossvaldata2 = np.array(train_data2)
    crossvaldata3 = np.array(train_data3)
    crossvaldata4 = np.array(train_data4)
    crossvallabels = np.array(labelstrain)

    #Define 10-fold cross validation test harness with StratifiedKFold
    skfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)

    #Define variabels to store the scores for every fold
    accscores = []
    precisionscores = []
    recallscores = []
    f1scores = []
    

    #Fold number, needed for stoing to csv
    nrfold = 1

    #Cross validation
    for train, test in skfold.split(train_data, train_labels):
        time_callback = TimeHistory() 
        
        foldmodel = clone_model(model)
        
        #evaluate the positive class (=ironic data)
        precision = km.binary_precision(label=1)
        recall = km.binary_recall(label=1) 
        f1 = km.f1_score(label=1)
        
        foldmodel.compile(loss = lossfunction,
              optimizer=optimizer,
              metrics=['accuracy',precision, recall, f1])

        #Train the model with validation data splited through cross validation
        hist= foldmodel.fit([crossvaldata[train], crossvaldata2[train],  crossvaldata3[train], crossvaldata4[train]], 
                            crossvallabels[train], epochs=nrepochs, batch_size=nrbatch, 
                            validation_data=([crossvaldata[test],crossvaldata2[test], crossvaldata3[test], crossvaldata4[test]], 
                            crossvallabels[test]), shuffle=False, verbose = 0, callbacks=[time_callback])

        #Append average of the validation scores for the current fold
        accscores.append(np.mean(hist.history['val_acc'])*100)
        precisionscores.append(np.mean(hist.history['val_precision'])*100)
        recallscores.append(np.mean(hist.history['val_recall'])*100)
        f1scores.append(np.mean(hist.history['val_f1_score'])*100)

        print("\n-----Fold "+str(nrfold)+"--------")
        
        #Plot the metrics for every epoch for the current fold
        from matplotlib import pyplot
        %matplotlib inline
        pyplot.figure(figsize=(15,10))
        pyplot.plot(hist.history['acc']) 
        pyplot.plot(hist.history['f1_score'])
        pyplot.plot(hist.history['val_acc']) 
        pyplot.plot(hist.history['val_f1_score'])
        #pyplot.plot(hist.history['loss'])
        pyplot.xticks(np.arange(1,nrepochs))
        pyplot.legend(['Train Accuracy', 'Train F1', 'Validation Accuracy', 'Validation F1', 'Loss'], loc=(1.04,0.5))
        pyplot.show()
        
        #Print the average validation scores for the current fold
        print("Average Accuracy: %.2f%%" % ((np.mean(hist.history['val_acc']))*100))
        print("Average Precision: %.2f%%" % ((np.mean(hist.history['val_precision']))*100))
        print("Average Recall: %.2f%%" % ((np.mean(hist.history['val_recall']))*100))
        print("Average F1: %.2f%%" % ((np.mean(hist.history['val_f1_score']))*100))

        #Store the keras model
        foldmodel.save(outputpath+str(datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S"))+'_'+outputname+
                       '_'+str(nrfold)+'_model.h5')

        #Store the train and validation scores for every epoch for the current fold to csv + train time
        f= open(outputpath+str(datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S"))+'_'+outputname+'_'+
                str(nrfold)+'_train_val_scores.csv','w+')
        f.write('Epoch; Loss; Accuracy; Precision; Recall; F1; Val_loss; Val_Accuracy; Val_Precision; VAl_Recall; Val_F1; Time \n')

        for i in range(len(hist.history['acc'])):
            f.write(str(i+1)+';'+str(hist.history['loss'][i])+';'+str(hist.history['acc'][i])+';'+
                    str(hist.history['precision'][i])+';'+str(hist.history['recall'][i])+';'+
                    str(hist.history['f1_score'][i])+';'+str(hist.history['val_loss'][i])+';'+
                    str(hist.history['val_acc'][i])+';'+str(hist.history['val_precision'][i])+';'+
                    str(hist.history['val_recall'][i])+';'+str(hist.history['val_f1_score'][i])+';'+
                    str(time_callback.times[i])+'\n')

        f.close()

        nrfold+=1
      
    #Plot and print the results of all folds 
    print("\n-------Overallresults-------")

    #Plot the overall Metrics
    pyplot.figure(figsize=(15,10))
    pyplot.plot(accscores) 
    pyplot.plot(precisionscores)
    pyplot.plot(recallscores)
    pyplot.plot(f1scores)
    #pyplot.xticks(np.arange(1,nrfold))
    pyplot.legend(['Accuracy', 'Precision', 'Recall', 'F1'], loc=(1.04,0.5))
    pyplot.show()

    #Print mean, min and max values
    print("Accuracy: Mean = %.2f%% (+/- %.2f%%), Max = %.2f%%, Min =  %.2f%%" % (np.mean(accscores), np.std(accscores), np.max(accscores), np.min(accscores)))
    print("Precision: Mean = %.2f%% (+/- %.2f%%), Max = %.2f%%, Min =  %.2f%%" % (np.mean(precisionscores), np.std(precisionscores), np.max(precisionscores), np.min(precisionscores)))
    print("Recall: Mean = %.2f%% (+/- %.2f%%), Max = %.2f%%, Min =  %.2f%%" % (np.mean(recallscores), np.std(recallscores), np.max(recallscores), np.min(recallscores)))
    print("F1: Mean = %.2f%% (+/- %.2f%%), Max = %.2f%%, Min =  %.2f%%" % (np.mean(f1scores), np.std(f1scores), np.max(f1scores), np.min(f1scores)))
    
    #Write validation scores for every fold to csv
    f= open(outputpath+str(datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S"))+' '+outputname+
            '_validationscores.csv',"w+")
    f.write('Accuracy; Precision; Recall; F1 \n')
    for i in range(len(accscores)):
        f.write(str(accscores[i])+';'+str(precisionscores[i])+';'+str(recallscores[i])+';'+str(f1scores[i])+'\n')

    f.close()

In [ ]:
#model = Kearas model
#softmax = boolean, true if the model uses softmaxe in the output layer otherwise false
#outputpath = path were the output csv should be stored
#outputname = name to use for the output csv
def train_evaluate_final_model(model, nrepochs, nrbatch, optimizer, lossfunction, train_data, train_labels, test_data, test_labels, softmax, outputpath, outputname):
    import datetime
    
    seed = 7
    np.random.seed(seed)
    tf.set_random_seed(seed)
    rd.seed(seed)
    
    #needed for softmax-activation
    if softmax:
        labelstrain = to_categorical(train_labels)
        labelstest = to_categorical(test_labels)
    else:
        labelstrain = train_labels
        labelstest = test_labels

    time_callback = TimeHistory() 
    
    #evaluate the positive class (=ironic data)
    precision =km.binary_precision(label=1)
    recall = km.binary_recall(label=1)
    f1 = km.f1_score(label=1)

    model.compile(loss = lossfunction,
          optimizer=optimizer,
          metrics=['accuracy', precision, recall, f1])

    #Train the model 
    hist= model.fit(train_data, labelstrain, epochs=nrepochs, batch_size=nrbatch, shuffle=True, verbose = 0, callbacks=[time_callback])

    from matplotlib import pyplot
    %matplotlib inline
    pyplot.figure(figsize=(15,10))
    pyplot.plot(hist.history['acc']) 
    pyplot.plot(hist.history['f1_score'])
    #pyplot.plot(hist.history['loss'])
    pyplot.xticks(np.arange(1,nrepochs))
    pyplot.legend(['Train Accuracy', 'Train F1', 'Validation Accuracy', 'Validation F1', 'Loss'], loc=(1.04,0.5))
    pyplot.show()

    #Store the keras model
    model.save(outputpath+str(datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S"))+'_'+outputname+
                   '_'+'_model.h5')        
    
    score = model.evaluate(test_data, labelstest, verbose = 0)
    
    print("\n-----Test Scores----")
    print("Loss: " + str(score[0]))
    print("Accuracy: " + str(score[1]*100))
    print("Precision: " + str(score[2]*100))
    print("Recall: " + str(score[3]*100))
    print("F1: " + str(score[4]*100))


    #stores the test results        
    f= open(outputpath+str(datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S"))+' '+outputname+'_finaltestresults.csv','w+')
    f.write('Loss;'+str(score[0])+'\n'+
            'Accuracy;'+str(score[1])+'\n'+
            'Precision;'+str(score[2])+'\n'+
            'Recall;'+str(score[3])+'\n'+
            'F1;'+str(score[4])+'\n')
    f.close() 

In [ ]:
#model = Kearas model
#softmax = boolean, true if the model uses softmaxe in the output layer otherwise false
#outputpath = path were the output csv should be stored
#outputname = name to use for the output csv
def train_evaluate_final_model_with_es(model, nrepochs, nrbatch, optimizer, lossfunction, train_data, train_labels, test_data, test_labels, softmax, outputpath, outputname, es_monitor, es_patience):
    import datetime
    
    seed = 7
    np.random.seed(seed)
    tf.set_random_seed(seed)
    rd.seed(seed)
    
    #needed for softmax-activation
    if softmax:
        labelstrain = to_categorical(train_labels)
    else:
        labelstrain = train_labels

    time_callback = TimeHistory() 
    
    #evaluate the positive class (=ironic data)
    precision =km.binary_precision(label=1)
    recall = km.binary_recall(label=1)
    f1 = km.f1_score(label=1)

    model.compile(loss = lossfunction,
          optimizer=optimizer,
          metrics=['accuracy', precision, recall, f1])
    
    es = EarlyStopping(monitor=es_monitor,
                              min_delta=0,
                              patience=es_patience,
                              verbose=0, mode='auto')

    #Train the model 
    hist= model.fit(train_data, labelstrain, validation_split = 0.2, epochs=nrepochs, batch_size=nrbatch, shuffle=True, verbose = 1, callbacks=[es, time_callback])

    from matplotlib import pyplot
    %matplotlib inline
    pyplot.figure(figsize=(15,10))
    pyplot.plot(hist.history['acc']) 
    pyplot.plot(hist.history['f1_score'])
    #pyplot.plot(hist.history['loss'])
    pyplot.xticks(np.arange(1,nrepochs))
    pyplot.legend(['Train Accuracy', 'Train F1', 'Validation Accuracy', 'Validation F1', 'Loss'], loc=(1.04,0.5))
    pyplot.show()

    #Store the keras model
    model.save(outputpath+str(datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S"))+'_'+outputname+
                   '_'+'_model.h5')
    
    if softmax:
        labelstest = to_categorical(test_labels)
    else:
        labelstest = test_labels
    
    score = model.evaluate(test_data, labelstest, verbose = 0)
    
    print("\n-----Test Scores----")
    print("Loss: " + str(score[0]))
    print("Accuracy: " + str(score[1]*100))
    print("Precision: " + str(score[2]*100))
    print("Recall: " + str(score[3]*100))
    print("F1: " + str(score[4]*100))


    #stores the test results        
    f= open(outputpath+str(datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S"))+' '+outputname+'_finaltestresults.csv','w+')
    f.write('Loss;'+str(score[0])+'\n'+
            'Accuracy;'+str(score[1])+'\n'+
            'Precision;'+str(score[2])+'\n'+
            'Recall;'+str(score[3])+'\n'+
            'F1;'+str(score[4])+'\n')
    f.close() 

In [ ]:
from keras.callbacks import TerminateOnNaN
#model = Kearas model
#softmax = boolean, true if the model uses softmaxe in the output layer otherwise false
#outputpath = path were the output csv should be stored
#outputname = name to use for the output csv
def train_evaluate_three_inputs_with_terminate(model, nrepochs, nrbatch, optimizer, lossfunction, train_data, train_data2, train_data3,
                                train_labels, test_data, test_data2, test_data3, test_labels, softmax, outputpath, outputname):
    import datetime
    
    seed = 7
    np.random.seed(seed)
    tf.set_random_seed(seed)
    rd.seed(seed)
    
    #needed for softmax-activation
    if softmax:
        labelstrain = to_categorical(train_labels)
        labelstest = to_categorical(test_labels)
    else:
        labelstrain = train_labels
        labelstest = test_labels

    time_callback = TimeHistory() 
    terminate = TerminateOnNaN()
    
    #evaluate the positive class (=ironic data)
    precision =km.binary_precision(label=1)
    recall = km.binary_recall(label=1)
    f1 = km.f1_score(label=1)

    model.compile(loss = lossfunction,
          optimizer=optimizer,
          metrics=['accuracy', precision, recall, f1])

    #Train the model 
    hist= model.fit([train_data, train_data2, train_data3], labelstrain, epochs=nrepochs, batch_size=nrbatch, 
                    shuffle=True, verbose = 0, callbacks=[time_callback, terminate])

    from matplotlib import pyplot
    %matplotlib inline
    pyplot.figure(figsize=(15,10))
    pyplot.plot(hist.history['acc']) 
    pyplot.plot(hist.history['f1_score'])
    #pyplot.plot(hist.history['loss'])
    pyplot.xticks(np.arange(1,nrepochs))
    pyplot.legend(['Train Accuracy', 'Train F1', 'Validation Accuracy', 'Validation F1', 'Loss'], loc=(1.04,0.5))
    pyplot.show()

    #Store the keras model
    model.save(outputpath+str(datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S"))+'_'+outputname+
                   '_'+'_model.h5')        
    
    score = model.evaluate([test_data,test_data2, test_data3], labelstest, verbose = 0)
    
    print("\n-----Test Scores----")
    print("Loss: " + str(score[0]))
    print("Accuracy: " + str(score[1]*100))
    print("Precision: " + str(score[2]*100))
    print("Recall: " + str(score[3]*100))
    print("F1: " + str(score[4]*100))


    #stores the test results        
    f= open(outputpath+str(datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S"))+' '+outputname+'_finaltestresults.csv','w+')
    f.write('Loss;'+str(score[0])+'\n'+
            'Accuracy;'+str(score[1])+'\n'+
            'Precision;'+str(score[2])+'\n'+
            'Recall;'+str(score[3])+'\n'+
            'F1;'+str(score[4])+'\n')
    f.close() 

In [ ]:
#model = Kearas model
#softmax = boolean, true if the model uses softmaxe in the output layer otherwise false
#outputpath = path were the output csv should be stored
#outputname = name to use for the output csv
def train_evaluate_two_inputs(model, nrepochs, nrbatch, optimizer, lossfunction, train_data, train_data2, train_labels, 
                              test_data, test_data2, test_labels, softmax, outputpath, outputname):
    import datetime
    
    seed = 7
    np.random.seed(seed)
    tf.set_random_seed(seed)
    rd.seed(seed)
    
    #needed for softmax-activation
    if softmax:
        labelstrain = to_categorical(train_labels)
        labelstest = to_categorical(test_labels)
    else:
        labelstrain = train_labels
        labelstest = test_labels

    time_callback = TimeHistory() 
    
    #evaluate the positive class (=ironic data)
    precision =km.binary_precision(label=1)
    recall = km.binary_recall(label=1)
    f1 = km.f1_score(label=1)

    model.compile(loss = lossfunction,
          optimizer=optimizer,
          metrics=['accuracy', precision, recall, f1])

    #Train the model 
    hist= model.fit([train_data, train_data2], labelstrain, epochs=nrepochs, batch_size=nrbatch, shuffle=True, verbose = 0, 
                    callbacks=[time_callback])

    from matplotlib import pyplot
    %matplotlib inline
    pyplot.figure(figsize=(15,10))
    pyplot.plot(hist.history['acc']) 
    pyplot.plot(hist.history['f1_score'])
    #pyplot.plot(hist.history['loss'])
    pyplot.xticks(np.arange(1,nrepochs))
    pyplot.legend(['Train Accuracy', 'Train F1', 'Validation Accuracy', 'Validation F1', 'Loss'], loc=(1.04,0.5))
    pyplot.show()

    #Store the keras model
    model.save(outputpath+str(datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S"))+'_'+outputname+
                   '_'+'_model.h5')        
    
    score = model.evaluate([test_data,test_data2], labelstest, verbose = 0)
    
    print("\n-----Test Scores----")
    print("Loss: " + str(score[0]))
    print("Accuracy: " + str(score[1]*100))
    print("Precision: " + str(score[2]*100))
    print("Recall: " + str(score[3]*100))
    print("F1: " + str(score[4]*100))


    #stores the test results        
    f= open(outputpath+str(datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S"))+' '+outputname+'_finaltestresults.csv','w+')
    f.write('Loss;'+str(score[0])+'\n'+
            'Accuracy;'+str(score[1])+'\n'+
            'Precision;'+str(score[2])+'\n'+
            'Recall;'+str(score[3])+'\n'+
            'F1;'+str(score[4])+'\n')
    f.close() 

In [ ]:
#model = Kearas model
#softmax = boolean, true if the model uses softmaxe in the output layer otherwise false
#outputpath = path were the output csv should be stored
#outputname = name to use for the output csv
def train_evaluate_three_inputs(model, nrepochs, nrbatch, optimizer, lossfunction, train_data, train_data2, train_data3,
                                train_labels, test_data, test_data2, test_data3, test_labels, softmax, outputpath, outputname):
    import datetime
    
    seed = 7
    np.random.seed(seed)
    tf.set_random_seed(seed)
    rd.seed(seed)
    
    #needed for softmax-activation
    if softmax:
        labelstrain = to_categorical(train_labels)
        labelstest = to_categorical(test_labels)
    else:
        labelstrain = train_labels
        labelstest = test_labels

    time_callback = TimeHistory() 
    
    #evaluate the positive class (=ironic data)
    precision =km.binary_precision(label=1)
    recall = km.binary_recall(label=1)
    f1 = km.f1_score(label=1)

    model.compile(loss = lossfunction,
          optimizer=optimizer,
          metrics=['accuracy', precision, recall, f1])

    #Train the model 
    hist= model.fit([train_data, train_data2, train_data3], labelstrain, epochs=nrepochs, batch_size=nrbatch, 
                    shuffle=True, verbose = 0, callbacks=[time_callback])

    from matplotlib import pyplot
    %matplotlib inline
    pyplot.figure(figsize=(15,10))
    pyplot.plot(hist.history['acc']) 
    pyplot.plot(hist.history['f1_score'])
    #pyplot.plot(hist.history['loss'])
    pyplot.xticks(np.arange(1,nrepochs))
    pyplot.legend(['Train Accuracy', 'Train F1', 'Validation Accuracy', 'Validation F1', 'Loss'], loc=(1.04,0.5))
    pyplot.show()

    #Store the keras model
    model.save(outputpath+str(datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S"))+'_'+outputname+
                   '_'+'_model.h5')        
    
    score = model.evaluate([test_data,test_data2, test_data3], labelstest, verbose = 0)
    
    print("\n-----Test Scores----")
    print("Loss: " + str(score[0]))
    print("Accuracy: " + str(score[1]*100))
    print("Precision: " + str(score[2]*100))
    print("Recall: " + str(score[3]*100))
    print("F1: " + str(score[4]*100))


    #stores the test results        
    f= open(outputpath+str(datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S"))+' '+outputname+'_finaltestresults.csv','w+')
    f.write('Loss;'+str(score[0])+'\n'+
            'Accuracy;'+str(score[1])+'\n'+
            'Precision;'+str(score[2])+'\n'+
            'Recall;'+str(score[3])+'\n'+
            'F1;'+str(score[4])+'\n')
    f.close() 

In [ ]:
#model = Kearas model
#softmax = boolean, true if the model uses softmaxe in the output layer otherwise false
#outputpath = path were the output csv should be stored
#outputname = name to use for the output csv
def train_evaluate_four_inputs(model, nrepochs, nrbatch, optimizer, lossfunction, train_data, train_data2, train_data3, train_data4,
                                train_labels, test_data, test_data2, test_data3, test_data4, test_labels, softmax, outputpath, outputname):
    import datetime
    
    seed = 7
    np.random.seed(seed)
    tf.set_random_seed(seed)
    rd.seed(seed)
    
    #needed for softmax-activation
    if softmax:
        labelstrain = to_categorical(train_labels)
        labelstest = to_categorical(test_labels)
    else:
        labelstrain = train_labels
        labelstest = test_labels

    time_callback = TimeHistory() 
    
    #evaluate the positive class (=ironic data)
    precision =km.binary_precision(label=1)
    recall = km.binary_recall(label=1)
    f1 = km.f1_score(label=1)

    model.compile(loss = lossfunction,
          optimizer=optimizer,
          metrics=['accuracy', precision, recall, f1])

    #Train the model 
    hist= model.fit([train_data, train_data2, train_data3, train_data4], labelstrain, epochs=nrepochs, batch_size=nrbatch, 
                    shuffle=True, verbose = 0, callbacks=[time_callback])

    from matplotlib import pyplot
    %matplotlib inline
    pyplot.figure(figsize=(15,10))
    pyplot.plot(hist.history['acc']) 
    pyplot.plot(hist.history['f1_score'])
    #pyplot.plot(hist.history['loss'])
    pyplot.xticks(np.arange(1,nrepochs))
    pyplot.legend(['Train Accuracy', 'Train F1', 'Validation Accuracy', 'Validation F1', 'Loss'], loc=(1.04,0.5))
    pyplot.show()

    #Store the keras model
    model.save(outputpath+str(datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S"))+'_'+outputname+
                   '_'+'_model.h5')        
    
    score = model.evaluate([test_data,test_data2, test_data3, test_data4], labelstest, verbose = 0)
    
    print("\n-----Test Scores----")
    print("Loss: " + str(score[0]))
    print("Accuracy: " + str(score[1]*100))
    print("Precision: " + str(score[2]*100))
    print("Recall: " + str(score[3]*100))
    print("F1: " + str(score[4]*100))


    #stores the test results        
    f= open(outputpath+str(datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S"))+' '+outputname+'_finaltestresults.csv','w+')
    f.write('Loss;'+str(score[0])+'\n'+
            'Accuracy;'+str(score[1])+'\n'+
            'Precision;'+str(score[2])+'\n'+
            'Recall;'+str(score[3])+'\n'+
            'F1;'+str(score[4])+'\n')
    f.close() 

In [ ]:
#Evaluate the model with the held out train data
def test_model(filename, testdata, test_labels, softmax, outputpath, outputname):
    import datetime 
    finalmodel = load_model(str(filename), custom_objects={'binary_precision':km.binary_precision(label=1), 'binary_recall': km.binary_recall(label=1), 'binary_f1_score':km.f1_score(label=1), 'Attention':Attention()})

    #needed for softmax-activation
    if softmax:
        labelstest = to_categorical(test_labels)
    else:
        labelstest = test_labels
        
    score = finalmodel.evaluate(testdata, labelstest, verbose = 0)
    
    print("\n-----Test Scores----")
    print("Loss: " + str(score[0]))
    print("Accuracy: " + str(score[1]*100))
    print("Precision: " + str(score[2]*100))
    print("Recall: " + str(score[3]*100))
    print("F1: " + str(score[4]*100))


    #stores the test results        
    f= open(outputpath+str(datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S"))+' '+outputname+'_finaltestresults.csv','w+')
    f.write('Loss;'+str(score[0])+'\n'+
            'Accuracy;'+str(score[1])+'\n'+
            'Precision;'+str(score[2])+'\n'+
            'Recall;'+str(score[3])+'\n'+
            'F1;'+str(score[4])+'\n')
    f.close() 